In [1]:
import numpy as np

class SimpleRNN:
    """
    这是一个最简循环神经网络 (RNN) 的实现示例，用于演示基本的前向传播和反向传播流程。
    """

    def __init__(self, input_dim, hidden_dim, output_dim, lr=0.01):
        """
        RNN 参数初始化

        参数：
        - input_dim:  输入数据的维度
        - hidden_dim: 隐藏层维度
        - output_dim: 输出层维度
        - lr:         学习率（learning rate）
        """
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lr = lr

        # 初始化权重：输入 -> 隐藏
        # shape = (input_dim, hidden_dim)
        self.W_xh = np.random.randn(input_dim, hidden_dim) * 0.01

        # 初始化权重：隐藏 -> 隐藏（即循环部分）
        # shape = (hidden_dim, hidden_dim)
        self.W_hh = np.random.randn(hidden_dim, hidden_dim) * 0.01

        # 初始化权重：隐藏 -> 输出
        # shape = (hidden_dim, output_dim)
        self.W_hy = np.random.randn(hidden_dim, output_dim) * 0.01

        # 初始化偏置：隐藏层和输出层
        # shape = (hidden_dim,)
        self.b_h = np.zeros((hidden_dim, ))
        # shape = (output_dim,)
        self.b_y = np.zeros((output_dim, ))

    def forward(self, x_seq):
        """
        RNN 的前向传播。
        给定序列 x_seq（形状 (T, input_dim)），
        依次计算每个时间步 t 的隐藏状态 h(t) 和输出 y(t)。

        返回:
        - h_seq: 每个时间步的隐藏状态数组 (T, hidden_dim)
        - y_seq: 每个时间步的输出数组 (T, output_dim)
        """
        T = x_seq.shape[0]  # 序列长度
        # 初始化隐藏状态 h(0)，设为 0 向量
        h_t = np.zeros((self.hidden_dim, ))

        # 用来收集每个时间步的隐藏状态和输出
        h_seq = []
        y_seq = []

        for t in range(T):
            # 取出第 t 个时间步的输入 x(t)，形状 (input_dim, )
            x_t = x_seq[t]  # shape = (input_dim, )

            # 1) 计算隐藏状态 h(t) = tanh(x(t) * W_xh + h(t-1) * W_hh + b_h)
            h_t = np.tanh(
                x_t @ self.W_xh +
                h_t @ self.W_hh +
                self.b_h
            )
            h_seq.append(h_t)

            # 2) 计算输出 y(t) = h(t) * W_hy + b_y
            y_t = h_t @ self.W_hy + self.b_y
            y_seq.append(y_t)

        # 转成 numpy 数组
        h_seq = np.stack(h_seq, axis=0)  # (T, hidden_dim)
        y_seq = np.stack(y_seq, axis=0)  # (T, output_dim)

        return h_seq, y_seq

    def compute_loss(self, y_pred, y_true):
        """
        使用均方误差 (MSE) 计算损失。

        参数：
        - y_pred: 模型预测输出，形状 (T, output_dim)
        - y_true: 真实目标输出，形状 (T, output_dim)

        返回:
        - MSE 损失值 (float)
        """
        return 0.5 * np.sum((y_pred - y_true) ** 2)

    def backward(self, x_seq, h_seq, y_seq, y_true):
        """
        通过时间的反向传播 (BPTT)，计算并更新各参数梯度。

        参数：
        - x_seq:  输入序列 (T, input_dim)
        - h_seq:  每个时间步的隐藏状态 (T, hidden_dim)
        - y_seq:  每个时间步的预测输出 (T, output_dim)
        - y_true: 每个时间步的真实输出 (T, output_dim)
        """
        T = x_seq.shape[0]

        # 初始化各参数的梯度
        dW_xh = np.zeros_like(self.W_xh)
        dW_hh = np.zeros_like(self.W_hh)
        dW_hy = np.zeros_like(self.W_hy)
        db_h  = np.zeros_like(self.b_h)
        db_y  = np.zeros_like(self.b_y)

        # dh_next 用于存放来自后面时间步的隐藏状态梯度
        dh_next = np.zeros((self.hidden_dim, ))

        # 从最后一个时间步往前进行 BPTT
        for t in reversed(range(T)):
            # ============== 第 1 步：计算输出层梯度 ==============
            dy = (y_seq[t] - y_true[t])  # shape = (output_dim, )
            # 对 W_hy 和 b_y 的梯度
            dW_hy += np.outer(h_seq[t], dy)  # (hidden_dim, ) -> (hidden_dim, output_dim)
            db_y  += dy                      

            # ============== 第 2 步：累加对隐藏状态的梯度 ==============
            dh = dy @ self.W_hy.T + dh_next  # (hidden_dim, )
            
            # 由于 h(t) = tanh(z)，z = x(t)*W_xh + h(t-1)*W_hh + b_h
            # tanh'(z) = 1 - tanh^2(z)
            # 因此对 z 的梯度为 dh * (1 - h(t)^2)
            dtanh = (1 - h_seq[t]**2) * dh

            # ============== 第 3 步：计算对参数的梯度 ==============
            # (1) 对 W_xh 的梯度
            x_t = x_seq[t]  # shape = (input_dim, )
            dW_xh += np.outer(x_t, dtanh)  # (input_dim, hidden_dim)

            # (2) 对 W_hh 的梯度
            h_prev = np.zeros_like(h_seq[t]) if t == 0 else h_seq[t-1]
            dW_hh += np.outer(h_prev, dtanh)  # (hidden_dim, hidden_dim)

            # (3) 对偏置 b_h 的梯度
            db_h  += dtanh  # shape = (hidden_dim, )

            # ============== 第 4 步：计算对前一个隐藏状态的梯度 ==============
            dh_next = dtanh @ self.W_hh.T  # shape = (hidden_dim, )

        # ============== 第 5 步：参数更新 (梯度下降) ==============
        self.W_xh -= self.lr * dW_xh
        self.W_hh -= self.lr * dW_hh
        self.W_hy -= self.lr * dW_hy
        self.b_h  -= self.lr * db_h
        self.b_y  -= self.lr * db_y

    def train_step(self, x_seq, y_true):
        """
        进行一次完整的前向传播、损失计算和反向传播，返回当前损失值。

        参数：
        - x_seq:  输入序列 (T, input_dim)
        - y_true: 目标序列 (T, output_dim)

        返回:
        - 当前的损失值 (float)
        """
        # 1) 前向传播
        h_seq, y_seq = self.forward(x_seq)

        # 2) 计算损失
        loss = self.compute_loss(y_seq, y_true)

        # 3) 反向传播
        self.backward(x_seq, h_seq, y_seq, y_true)

        return loss

# =========== 使用示例 ===========

if __name__ == "__main__":
    # 设置随机种子，便于复现实验结果
    np.random.seed(42)

    # 设定超参数
    T = 50           # 序列长度
    input_dim = 10   # 输入维度
    hidden_dim = 16  # 隐藏层维度（可自行调整）
    output_dim = 1   # 输出层维度（此处假设做回归或一个数值的预测）
    lr = 0.01        # 学习率
    num_epochs = 100 # 训练轮数

    # 构造随机输入和目标
    # x_data: (T, input_dim)
    x_data = np.random.randn(T, input_dim)
    # 假设目标 y_data = x_data 在第一个维度上的平均值（或其他简单映射），仅作示例
    y_data = np.mean(x_data, axis=1, keepdims=True)  # shape = (T, 1)

    # 初始化 RNN
    rnn = SimpleRNN(input_dim, hidden_dim, output_dim, lr=lr)

    # 训练
    for epoch in range(num_epochs):
        loss_val = rnn.train_step(x_data, y_data)
        if (epoch+1) % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Loss = {loss_val:.6f}")

    # 最终预测
    _, y_pred = rnn.forward(x_data)
    print("\n实际输出 y_data[:10]:")
    print(y_data[:10].flatten())
    print("预测输出 y_pred[:10]:")
    print(y_pred[:10].flatten())


Epoch 10/100, Loss = 2.276755
Epoch 20/100, Loss = 0.747057
Epoch 30/100, Loss = 0.065337
Epoch 40/100, Loss = 0.021704
Epoch 50/100, Loss = 0.009755
Epoch 60/100, Loss = 0.004976
Epoch 70/100, Loss = 0.002936
Epoch 80/100, Loss = 0.002053
Epoch 90/100, Loss = 0.001667
Epoch 100/100, Loss = 0.001493

实际输出 y_data[:10]:
[ 0.44806111 -0.79065823 -0.22184356 -0.31010667 -0.25282217  0.19944143
 -0.01505699 -0.04773612  0.10144743 -0.14919139]
预测输出 y_pred[:10]:
[ 0.45157019 -0.75982746 -0.22614844 -0.32124421 -0.25618755  0.2076185
 -0.0197995  -0.04907385  0.10088893 -0.15778372]
